# LLM agent Hackathon 
## Alireza Ghafarollahi, MIT

# AG2: Building LLM-powered multi-agent systems

In [ ]:
!pip install ag2
!pip install openai

In [5]:
import json
import os

with open('config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config["api_key"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Example: Group chat to assist the user with a tech-related query

## Agents

- ## coordinator agent

- ## technical support agent

- ## general support agent

## Tools

- ## Classify_query

## Context variables

# Enhance our system with Handoffs:

## We will enhance our support system with multiple specialized agents that handle different aspects of customer service:

- ## A coordinator agent that analyzes initial queries
- ## A general support agent for non-technical questions
- ## Specialists for different device types (computers and smartphones)
- ## An advanced troubleshooting agent for persistent issues

## This example demonstrates how a real-world support system might use handoffs to route customers efficiently.

In [32]:
from typing import Annotated
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern
from autogen.agentchat.group import (
    ContextVariables, ReplyResult, AgentTarget,
    OnCondition, StringLLMCondition,
    OnContextCondition, ExpressionContextCondition, ContextExpression,
    RevertToUserTarget
)

# Initialize context variables for our support system

In [33]:
support_context = ContextVariables(data={
    "query_count": 0,
    "repeat_issue": False,
    "previous_solutions": [],
    "issue_type": "",
    "issue_subtype": "",
})

# Create all our support agents

In [34]:
# Configure the LLM
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

# Create all our support agents
with llm_config:
    # Main coordinator agent - the starting point for all user queries
    coordinator_agent = ConversableAgent(
        name="coordinator_agent",
        system_message="""You are a support coordinator agent. Your role is to:
        1. Determine if a query is technical or general
        2. Use the classify_query function to route appropriately

        Do not attempt to solve issues yourself - your job is proper routing."""
    )

    # General support for non-technical questions
    general_agent = ConversableAgent(
        name="general_agent",
        system_message="""You are a general support agent who handles non-technical questions.
        If the user is a premium customer (check account_tier context variable),
        you should transfer them directly to the premium support agent.
        Otherwise, provide helpful responses to general inquiries."""
    )

    # Tech agent for initial technical assessment
    tech_agent = ConversableAgent(
        name="tech_agent",
        system_message="""You are a technical support agent who handles the initial assessment
        of all technical issues.

        If the user is a premium customer (check account_tier context variable),
        you should transfer them directly to the premium support agent.

        Otherwise, determine if the issue is related to:
        - Computer issues (laptops, desktops, PCs, Macs)
        - Smartphone issues (iPhones, Android phones, tablets)

        And route to the appropriate specialist."""
    )

    # Device-specific agents
    computer_agent = ConversableAgent(
        name="computer_agent",
        system_message="""You are a computer specialist who handles issues with laptops, desktops,
        PCs, and Macs. You provide troubleshooting for hardware and software issues specific to
        computers. You're knowledgeable about Windows, macOS, Linux, and common computer peripherals.

        For first-time issues, provide a solution directly.

        If a user returns and says they tried your solution but are still having the issue,
        use the check_repeat_issue function to escalate to advanced troubleshooting. Do not provide a solution
        yourself for returning users, simply route it to advanced troubleshooting."""
    )

    smartphone_agent = ConversableAgent(
        name="smartphone_agent",
        system_message="""You are a smartphone specialist who handles issues with mobile devices
        including iPhones, Android phones, and tablets. You're knowledgeable about iOS, Android,
        mobile apps, battery issues, screen problems, and connectivity troubleshooting.

        For first-time issues, provide a solution directly.

        If a user returns and says they tried your solution but are still having the issue,
        use the check_repeat_issue function to escalate to advanced troubleshooting. Do not provide a solution
        yourself for returning users, simply route it to advanced troubleshooting"""
    )

    # Advanced troubleshooting for complex issues
    advanced_troubleshooting_agent = ConversableAgent(
        name="advanced_troubleshooting_agent",
        system_message="""You are an advanced troubleshooting specialist who handles complex,
        persistent issues that weren't resolved by initial solutions. You provide deeper
        diagnostic approaches and more comprehensive solutions for difficult technical problems."""
    )

# Create tool functions with Context Variables and Handoffs

In [35]:
def classify_query(
    query: Annotated[str, "The user query to classify"],
    context_variables: ContextVariables
) -> ReplyResult:
    """Classify a user query and route to the appropriate agent."""
    # Update query count
    context_variables["query_count"] += 1

    # Simple classification logic
    technical_keywords = ["error", "bug", "broken", "crash", "not working", "shutting down",
                         "frozen", "blue screen", "won't start", "slow", "virus"]

    if any(keyword in query.lower() for keyword in technical_keywords):
        return ReplyResult(
            message="This appears to be a technical issue. Let me route you to our tech support team.",
            target=AgentTarget(tech_agent),
            context_variables=context_variables
        )
    else:
        return ReplyResult(
            message="This appears to be a general question. Let me connect you with our general support team.",
            target=AgentTarget(general_agent),
            context_variables=context_variables
        )

def check_repeat_issue(
    description: Annotated[str, "User's description of the continuing issue"],
    context_variables: ContextVariables
) -> ReplyResult:
    """Check if this is a repeat of an issue that wasn't resolved."""
    # Mark this as a repeat issue in the context
    context_variables["repeat_issue"] = True
    context_variables["continuing_issue"] = description

    return ReplyResult(
        message="I understand that your issue wasn't resolved. Let me connect you with our advanced troubleshooting specialist.",
        target=AgentTarget(advanced_troubleshooting_agent),
        context_variables=context_variables
    )

# Add tool functions to the appropriate agents

In [36]:
coordinator_agent.functions = [classify_query]
computer_agent.functions = [check_repeat_issue]
smartphone_agent.functions = [check_repeat_issue]

# Configuring LLM-Based Handoffs

In [37]:
tech_agent.handoffs

Handoffs(context_conditions=[], llm_conditions=[], after_work=None)

In [38]:
# Route based on device type using LLM-based handoffs
tech_agent.handoffs.add_llm_conditions([
    OnCondition(
        target=AgentTarget(computer_agent),
        condition=StringLLMCondition(prompt="Route to computer specialist when the issue involves laptops, desktops, PCs, or Macs."),
    ),
    OnCondition(
        target=AgentTarget(smartphone_agent),
        condition=StringLLMCondition(prompt="Route to smartphone specialist when the issue involves phones, mobile devices, iOS, or Android."),
    )
])

Handoffs(context_conditions=[], llm_conditions=[OnCondition(target=AgentTarget(agent_name='computer_agent'), condition=StringLLMCondition(prompt='Route to computer specialist when the issue involves laptops, desktops, PCs, or Macs.'), available=None, llm_function_name=None), OnCondition(target=AgentTarget(agent_name='smartphone_agent'), condition=StringLLMCondition(prompt='Route to smartphone specialist when the issue involves phones, mobile devices, iOS, or Android.'), available=None, llm_function_name=None)], after_work=None)

# Configuring Context-Based Handoffs

In [39]:
# Configure handoffs for computer agent - for repeat issues
computer_agent.handoffs.add_context_conditions([
    OnContextCondition(
        target=AgentTarget(advanced_troubleshooting_agent),
        condition=ExpressionContextCondition(
            expression=ContextExpression("${repeat_issue} == True")
        )
    )
])

# Similarly for smartphone agent
smartphone_agent.handoffs.add_context_conditions([
    OnContextCondition(
        target=AgentTarget(advanced_troubleshooting_agent),
        condition=ExpressionContextCondition(
            expression=ContextExpression("${repeat_issue} == True")
        )
    )
])

Handoffs(context_conditions=[OnContextCondition(target=AgentTarget(agent_name='advanced_troubleshooting_agent'), condition=ExpressionContextCondition(expression=ContextExpression(expression='${repeat_issue} == True')), available=None)], llm_conditions=[], after_work=None)

# Setting Default After-Work Behavior

In [40]:
# Configure handoffs for advanced troubleshooting agent
advanced_troubleshooting_agent.handoffs.set_after_work(RevertToUserTarget())

# General agent reverts to user after helping
general_agent.handoffs.set_after_work(RevertToUserTarget())

Handoffs(context_conditions=[], llm_conditions=[], after_work=RevertToUserTarget())

# Putting it all together

In [43]:
# Create the user agent
user = ConversableAgent(name="user", human_input_mode="ALWAYS")

# Set up the conversation pattern
pattern = AutoPattern(
    initial_agent=coordinator_agent,
    agents=[
        coordinator_agent,
        tech_agent,
        computer_agent,
        smartphone_agent,
        advanced_troubleshooting_agent,
        general_agent
    ],
    user_agent=user,
    context_variables=support_context,
    group_manager_args = {"llm_config": llm_config},
)

# Run the chat

In [44]:
result, final_context, last_agent = initiate_group_chat(
    pattern=pattern,
    messages="My laptop keeps shutting down randomly. Can you help?",
    max_rounds=15
)

user (to chat_manager):

My laptop keeps shutting down randomly. Can you help?

--------------------------------------------------------------------------------

Next speaker: coordinator_agent


>>>>>>>> USING AUTO REPLY...
coordinator_agent (to chat_manager):

This query appears to be technical in nature due to the issue with your laptop shutting down. I will route it accordingly. 

**Routing to technical support.**

--------------------------------------------------------------------------------

Next speaker: tech_agent


>>>>>>>> USING AUTO REPLY...
tech_agent (to chat_manager):

***** Suggested tool call (call_jlG51rIuybh3FQ4VbqacIKml): transfer_to_computer_agent_1 *****
Arguments: 
{}
*********************************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Group_Tool_Executor


>>>>>>>> EXECUTING FUNCTION transfer_to_computer_agent_1...
Call ID: call_jlG51rIuybh

Replying as user. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  I tried all your suggestions and none of them fixed my issue


user (to chat_manager):

I tried all your suggestions and none of them fixed my issue

--------------------------------------------------------------------------------

Next speaker: computer_agent


>>>>>>>> USING AUTO REPLY...
computer_agent (to chat_manager):

It sounds like you're still experiencing the same problem with your laptop. I will route you to advanced troubleshooting for further assistance. 

**Routing to advanced troubleshooting.**

--------------------------------------------------------------------------------

Next speaker: advanced_troubleshooting_agent


>>>>>>>> USING AUTO REPLY...
advanced_troubleshooting_agent (to chat_manager):

Thank you for your patience. Let's take a deeper look at the situation with your laptop. Since the initial steps didn't resolve the issue, we'll perform more targeted diagnostics to identify the root cause. Here are some advanced troubleshooting approaches:

### 1. **Check Event Viewer (Windows)**
The Event Viewer can provide insights i

Replying as user. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (fc9c1b25-1cef-41ca-afed-7f7c86da1fdb): User requested to end the conversation

>>>>>>>> TERMINATING RUN (9db5a495-4bcb-4d8e-8b26-71da4f7829be): No reply generated
